In [1]:
import mediapipe as mp
import cv2

In [2]:
mp_drawings = mp.solutions.drawing_utils
mp_drawings_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [113]:
# features = ['label']
# for i in ['left', 'right']:
#     for k in range(1, 22):
#         for j in ['x', 'y', 'z']:
#             features.append(f"{i}_{j}{k}")

In [157]:
# import csv
# with open('data.csv', 'w') as f:
#     csv_writer = csv.writer(f)
#     csv_writer.writerow(features)

In [3]:
def DataCreator(label):
    import csv
    mp_holistic = mp.solutions.holistic
    mp_drawings = mp.solutions.drawing_utils
    cap = cv2.VideoCapture(0)
    with mp_holistic.Holistic(
        min_detection_confidence = 0.5,
        min_tracking_confidence = 0.5) as holistic:
        left, right = None, None
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                print("Ignoring empty frame")
                continue

            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            results = holistic.process(image)

            right = results.right_hand_landmarks
            left = results.left_hand_landmarks
            right_hand_marks = [None]*63
            left_hand_marks = [None]*63

            if right is not None:
                ind = 0
                for l in right.landmark:
                    right_hand_marks[ind], right_hand_marks[ind + 1], right_hand_marks[ind + 2] = l.x, l.y, l.z
                    ind += 3

            if left is not None:
                ind = 0
                for l in left.landmark:
                    left_hand_marks[ind], left_hand_marks[ind + 1], left_hand_marks[ind + 2] = l.x, l.y, l.z
                    ind += 3
            
            # print(right_hand_marks)
            all_marks = [label] + left_hand_marks + right_hand_marks

            # Appending row
            with open('data.csv', mode = 'a', newline='') as f:
                csv_writer = csv.writer(f)
                csv_writer.writerow(all_marks)

            mp_drawings.draw_landmarks( image, 
                                            results.right_hand_landmarks,
                                            mp_holistic.HAND_CONNECTIONS,
                                            mp_drawings.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                            mp_drawings.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                    )
            mp_drawings.draw_landmarks( image, 
                                            results.left_hand_landmarks,
                                            mp_holistic.HAND_CONNECTIONS,
                                            mp_drawings.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                            mp_drawings.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                    )

            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            cv2.imshow('Holistic', cv2.flip(image, 1))

            if cv2.waitKey(5) & 0xFF == 27:
                break

    cap.release()
    cv2.destroyAllWindows()

In [4]:
def loadCsv(fname):
    import pandas as pd
    df = pd.read_csv(fname)
    return df.dropna(thresh=2).fillna(-2)

In [5]:
DataCreator('Father')

In [6]:
df = loadCsv('data.csv')

In [7]:
import pandas as pd
pd.DataFrame(df['label'].value_counts())

,label
nine,1716
eight,1705
One,1659
C,1467
B,1397
five,1377
two,1375
four,1339
six,1167
zero,1110
